# Pixiv Craw

> - ### import repos

In [1]:
import requests
import json
from tqdm import tqdm
from pixiv_crawler.downloader.downloader import Downloader

> - ### 按照artist的id来craw

1.按照画师的id进行爬取（后续会添加函数 按照排名，标签进行爬取）

In [2]:
def fetch_artist_works(artist_id, category):
    # pixiv 的请求返回一般有两种，一种是漫画，一种是插画，还有其他的我懒得搞了，毕竟用的人很少（笑）
    if category not in ["illusts", "manga"]:
        raise ValueError("Invalid category. Use 'illustrations' or 'manga'.")

    url = f"https://www.pixiv.net/ajax/user/{artist_id}/profile/all?lang=zh"

    additional_headers = {
        "x-user-id": "5640xxxx",
        "COOKIE": "first_visit_datetime_pc=2022-03-23+09%3A05%3A13; p_ab_id=2; p_ab_id_2=6; p_ab_d_id=496637752; yuid_b=FnRJY4g; _fbp=fb.1.1647993926346.868094683; privacy_policy_notification=0; a_type=0; b_type=1; _im_vid=01FYT7DA0M1MG08GJSVD70BA7M; adr_id=lTzPqgTf6xmkSZZddeH8LhLHDKfx8OJsbbCpSa4F8n6kESK0; login_ever=yes; _td=e09e84b3-b5d0-4284-8cbc-fe6a9605154a; _ga_692MXKN5XF=GS1.1.1665973029.1.0.1665973029.0.0.0; cto_bundle=MeOqNl9aSyUyRkklMkJFcFJ5WCUyQlNzZkozQTFYSVpESHZCRiUyRm83QWZ3aTdmNEVIbG9FRWkxUUlIbDl0eiUyQnZlRmU3SWo0d1VLMGFFMks1M0dXWnA4OFBhMkNhcjJGQ3Rjakl3JTJGc3lwcU5ZV1NiY0RMbSUyQmQ0aEVZY0J4TnM0RiUyRnBEJTJGcVJPeSUyQkwyWTAwUmFvNmQyV3R5QlRnN2RXSFFYUSUzRCUzRA; _im_uid.3929=i.HNTUXPphRC6Tos5lngHwCA; c_type=21; privacy_policy_agreement=6; __gads=ID=e032ebe84fc5b362:T=1696335939:RT=1696335939:S=ALNI_MYk8hTGJ0JVZ08XHPJ1chcTiPcYZQ; __gpi=UID=00000c56c30e862e:T=1696335939:RT=1696335939:S=ALNI_MZ0J6EZLNM_IHlenurDDcoh0qn7tw; _ga_ZQEKG3EF2C=GS1.1.1696335939.1.0.1696335941.58.0.0; __utmv=235335808.|2=login%20ever=yes=1^3=plan=normal=1^5=gender=male=1^6=user_id=56405112=1^9=p_ab_id=2=1^10=p_ab_id_2=6=1^11=lang=zh=1; _gcl_au=1.1.2068989696.1697797468; PHPSESSID=56405112_5F6NwBp0ibUH6FuPbdq8P8dKXRrJiakY; device_token=d9e66683eee3c683671ce6dd45961915; _ga_MZ1NL4PHH0=GS1.1.1698243034.3.1.1698243060.0.0.0; __utmz=235335808.1698489562.713.11.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); QSI_S_ZN_5hF4My7Ad6VNNAi=v:0:0; _gid=GA1.2.73441192.1699788507; _ga_ZPL8LPMDK3=GS1.1.1699930329.18.1.1699930357.0.0.0; __cf_bm=X67SFTFNe9KvEzK6AxS96ftfSVWEqcIgua.4.3EcMVc-1699935025-0-AfkUhf87pN2+siqG5r1YvGpphK/bajDCL09AvrYlnYhvxsux9sdfZQIN8W3Ha3ztxG0ErV0mnsKzNjCErKsxohNNrRGurEMD/KweXvYGAHlI; __utma=235335808.740189076.1647993924.1699930321.1699935028.744; __utmc=235335808; __utmt=1; cf_clearance=SJHsrjUq5DYhRK8AO2UWSsB.5pZbPWf68VmswfYjLVA-1699935773-0-1-631cd866.8b0b9526.341d8786-0.2.1699935773; _ga=GA1.2.740189076.1647993924; __utmb=235335808.16.10.1699935028; _ga_75BBYNYN9J=GS1.1.1699935028.495.1.1699936337.0.0.0"
    }

    if category == "illusts":
        additional_headers["Referer"] = f"https://www.pixiv.net/users/{artist_id}/illustrations"
    elif category == "manga":
        additional_headers["Referer"] = f"https://www.pixiv.net/users/{artist_id}/manga"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36",
    }

    headers.update(additional_headers)

    proxies = {
        "https": "127.0.0.1:7890"
    }

    response = requests.get(url, headers=headers, proxies=proxies, timeout=4)

    print(response.json())

    if response.status_code == 200:
        id_group = set(response.json()["body"][category].keys())
        return id_group

In [5]:
artist_id = 3094189
categorys = ["manga", "illusts"]
id = 0

id_group = set()

for category in categorys:
    id_group.update(fetch_artist_works(artist_id=artist_id, category=category))
    print(id_group)
    
artist_artworks_ids_path = "E:\\vscode_clustter\\code\\github\\Animate-Image-Bed\\utils\\crawler\\test\\jsons\\"

file_name = f"{artist_id}-artworksID-collect.json"

artist_dict = {artist_id: list(id_group)}

# 将字典保存到 JSON 文件中
with open(artist_artworks_ids_path + file_name, "w", encoding="utf-8") as json_file:
    json.dump(artist_dict, json_file, ensure_ascii=False, indent=4)



{'error': False, 'message': '', 'body': {'illusts': {'81225309': None, '78834571': None, '73521542': None, '70052155': None, '70024717': None, '69998496': None, '66355934': None, '64089360': None, '60474023': None, '58420472': None, '56471514': None, '54724406': None, '53859115': None, '51455457': None, '47749401': None, '44982128': None, '31957654': None}, 'manga': {'103686781': None, '100082808': None, '89532917': None, '81225213': None, '81225091': None, '78336518': None, '76053184': None, '72066662': None, '69998409': None, '66355845': None, '64089303': None, '60575412': None, '57988139': None, '53969876': None, '51907682': None, '44982929': None, '40466977': None, '37435957': None, '32202507': None}, 'novels': [], 'mangaSeries': [], 'novelSeries': [], 'pickup': [{'id': '103686781', 'title': 'C101新刊「性教育シリーズ総集編2」', 'illustType': 1, 'xRestrict': 1, 'restrict': 0, 'sl': 6, 'url': 'https://i.pximg.net/c/288x288_80_a2/img-master/img/2022/12/17/23/29/50/103686781_p0_square1200.jpg', 'des

1.2.按照tag进行爬取（后续会添加函数 按照排名，标签进行爬取）

In [3]:
def fetch_tag_works(tag_name, category, order):
    # pixiv 的请求返回一般有两种，一种是漫画，一种是插画，还有其他的我懒得搞了，毕竟用的人很少（笑）
    if category not in ["illust", "manga"]:
        raise ValueError("Invalid category. Use 'illustrations' or 'manga'.")

    """url = f"https://www.pixiv.net/ajax/search/illustrations/{tag_name}?"+ \ 
            "&asd" + \
            "lang=zh"""
    
    if category == "illust":
        category_rel = "illustrations"
    else:
        category_rel = "manga"
    
    url = "https://www.pixiv.net/ajax/search/{}/".format(category_rel) + \
            "{}?".format(tag_name) + \
            "&order={}".format("popular_d" if order else "date_d") + \
            f"&mode=all" + "&p=2&s_mode=s_tag&type=all&lang=zh"

    """
    /ajax/search/manga/BlueArchive?word=BlueArchive&order=date_d&mode=all&p=1&s_mode=s_tag_full&type=manga&lang=zh&version=6d99637327b03bc728989e9dfd2b67555a8dcbad HTTP/1.1
    """

    additional_headers = {
        "x-user-id": "5640xxxx",
        "COOKIE": "first_visit_datetime_pc=2022-03-23+09%3A05%3A13; p_ab_id=2; p_ab_id_2=6; p_ab_d_id=496637752; yuid_b=FnRJY4g; _fbp=fb.1.1647993926346.868094683; privacy_policy_notification=0; a_type=0; b_type=1; _im_vid=01FYT7DA0M1MG08GJSVD70BA7M; adr_id=lTzPqgTf6xmkSZZddeH8LhLHDKfx8OJsbbCpSa4F8n6kESK0; login_ever=yes; _td=e09e84b3-b5d0-4284-8cbc-fe6a9605154a; _ga_692MXKN5XF=GS1.1.1665973029.1.0.1665973029.0.0.0; cto_bundle=MeOqNl9aSyUyRkklMkJFcFJ5WCUyQlNzZkozQTFYSVpESHZCRiUyRm83QWZ3aTdmNEVIbG9FRWkxUUlIbDl0eiUyQnZlRmU3SWo0d1VLMGFFMks1M0dXWnA4OFBhMkNhcjJGQ3Rjakl3JTJGc3lwcU5ZV1NiY0RMbSUyQmQ0aEVZY0J4TnM0RiUyRnBEJTJGcVJPeSUyQkwyWTAwUmFvNmQyV3R5QlRnN2RXSFFYUSUzRCUzRA; _im_uid.3929=i.HNTUXPphRC6Tos5lngHwCA; c_type=21; privacy_policy_agreement=6; __gads=ID=e032ebe84fc5b362:T=1696335939:RT=1696335939:S=ALNI_MYk8hTGJ0JVZ08XHPJ1chcTiPcYZQ; __gpi=UID=00000c56c30e862e:T=1696335939:RT=1696335939:S=ALNI_MZ0J6EZLNM_IHlenurDDcoh0qn7tw; _ga_ZQEKG3EF2C=GS1.1.1696335939.1.0.1696335941.58.0.0; __utmv=235335808.|2=login%20ever=yes=1^3=plan=normal=1^5=gender=male=1^6=user_id=56405112=1^9=p_ab_id=2=1^10=p_ab_id_2=6=1^11=lang=zh=1; _gcl_au=1.1.2068989696.1697797468; PHPSESSID=56405112_5F6NwBp0ibUH6FuPbdq8P8dKXRrJiakY; device_token=d9e66683eee3c683671ce6dd45961915; _ga_MZ1NL4PHH0=GS1.1.1698243034.3.1.1698243060.0.0.0; __utmz=235335808.1698489562.713.11.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); QSI_S_ZN_5hF4My7Ad6VNNAi=v:0:0; _gid=GA1.2.73441192.1699788507; _ga_ZPL8LPMDK3=GS1.1.1699930329.18.1.1699930357.0.0.0; __cf_bm=X67SFTFNe9KvEzK6AxS96ftfSVWEqcIgua.4.3EcMVc-1699935025-0-AfkUhf87pN2+siqG5r1YvGpphK/bajDCL09AvrYlnYhvxsux9sdfZQIN8W3Ha3ztxG0ErV0mnsKzNjCErKsxohNNrRGurEMD/KweXvYGAHlI; __utma=235335808.740189076.1647993924.1699930321.1699935028.744; __utmc=235335808; __utmt=1; cf_clearance=SJHsrjUq5DYhRK8AO2UWSsB.5pZbPWf68VmswfYjLVA-1699935773-0-1-631cd866.8b0b9526.341d8786-0.2.1699935773; _ga=GA1.2.740189076.1647993924; __utmb=235335808.16.10.1699935028; _ga_75BBYNYN9J=GS1.1.1699935028.495.1.1699936337.0.0.0"
    }

    if category == "illusts":
        additional_headers["Referer"] = f"https://www.pixiv.net/tags/{tag_name}/illustrations"
    elif category == "manga":
        additional_headers["Referer"] = f"https://www.pixiv.net/tags/{tag_name}/manga"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36",
    }

    headers.update(additional_headers)

    proxies = {
        "https": "127.0.0.1:7890"
    }

    response = requests.get(url, headers=headers, proxies=proxies, timeout=4)

    print(response.json())

    json_content = response.json()

    # 指定保存路径和文件名
    save_path = "E://vscode_clustter//code//github//Animate-Image-Bed//utils//crawler//test//jsons//re.json"

    # 将 JSON 内容保存到文件
    with open(save_path, "w", encoding="utf-8") as json_file:
        json.dump(json_content, json_file, ensure_ascii=False, indent=4)

    if response.status_code == 200:
        response_data = response.json()["body"][category]["data"]
        id_group = set(entry["id"] for entry in response_data)
        return id_group

In [4]:
from urllib.parse import quote

# ...

tag_name = "竜華キサキ"
category = "illust"

id_group = set()

# Encode the tag_name before using it in the URL
encoded_tag_name = quote(tag_name, safe='')

id_group.update(fetch_tag_works(tag_name=encoded_tag_name, category=category, order=True))

print(len(id_group))


{'error': False, 'body': {'illust': {'data': [{'id': '113177269', 'title': '無題', 'illustType': 0, 'xRestrict': 1, 'restrict': 0, 'sl': 6, 'url': 'https://i.pximg.net/c/250x250_80_a2/img-master/img/2023/11/06/00/45/28/113177269_p0_square1200.jpg', 'description': '', 'tags': ['R-18', '竜華キサキ', 'ブルアカ', 'ブルーアーカイブ', 'ロリ'], 'userId': '55125022', 'userName': 'にほほひ', 'width': 737, 'height': 1141, 'pageCount': 2, 'isBookmarkable': True, 'bookmarkData': None, 'alt': '#竜華キサキ 無題 - にほほひ的插画', 'titleCaptionTranslation': {'workTitle': None, 'workCaption': None}, 'createDate': '2023-11-06T00:45:28+09:00', 'updateDate': '2023-11-06T00:45:28+09:00', 'isUnlisted': False, 'isMasked': False, 'aiType': 1, 'profileImageUrl': 'https://i.pximg.net/user-profile/img/2020/11/04/01/56/55/19612892_30ce6eccbd3f9824e99bb3b84e36a3f1_50.jpg'}, {'id': '113176748', 'title': 'Kisaki / 竜華キサキ', 'illustType': 0, 'xRestrict': 0, 'restrict': 0, 'sl': 2, 'url': 'https://i.pximg.net/c/250x250_80_a2/img-master/img/2023/11/06/00/30/

2.对爬取的图片id进行对应url的提取和集成

In [5]:
print("===== collector start =====")

downloader = Downloader(capacity=1024)

with tqdm(total=len(id_group), desc="collecting urls") as pbar:
    urls = [f"https://www.pixiv.net/ajax/illust/{illust_id}/pages?lang=zh"
            for illust_id in id_group]
    additional_headers = [
        {
            "Referer": f"https://www.pixiv.net/artworks/{illust_id}",
            "x-user-id": "5640xxxx",
            "COOKIE": "first_visit_datetime_pc=2022-03-23+09%3A05%3A13; p_ab_id=2; p_ab_id_2=6; p_ab_d_id=496637752; yuid_b=FnRJY4g; _fbp=fb.1.1647993926346.868094683; privacy_policy_notification=0; a_type=0; b_type=1; _im_vid=01FYT7DA0M1MG08GJSVD70BA7M; adr_id=lTzPqgTf6xmkSZZddeH8LhLHDKfx8OJsbbCpSa4F8n6kESK0; login_ever=yes; _td=e09e84b3-b5d0-4284-8cbc-fe6a9605154a; _ga_692MXKN5XF=GS1.1.1665973029.1.0.1665973029.0.0.0; cto_bundle=MeOqNl9aSyUyRkklMkJFcFJ5WCUyQlNzZkozQTFYSVpESHZCRiUyRm83QWZ3aTdmNEVIbG9FRWkxUUlIbDl0eiUyQnZlRmU3SWo0d1VLMGFFMks1M0dXWnA4OFBhMkNhcjJGQ3Rjakl3JTJGc3lwcU5ZV1NiY0RMbSUyQmQ0aEVZY0J4TnM0RiUyRnBEJTJGcVJPeSUyQkwyWTAwUmFvNmQyV3R5QlRnN2RXSFFYUSUzRCUzRA; _im_uid.3929=i.HNTUXPphRC6Tos5lngHwCA; c_type=21; privacy_policy_agreement=6; __gads=ID=e032ebe84fc5b362:T=1696335939:RT=1696335939:S=ALNI_MYk8hTGJ0JVZ08XHPJ1chcTiPcYZQ; __gpi=UID=00000c56c30e862e:T=1696335939:RT=1696335939:S=ALNI_MZ0J6EZLNM_IHlenurDDcoh0qn7tw; _ga_ZQEKG3EF2C=GS1.1.1696335939.1.0.1696335941.58.0.0; __utmv=235335808.|2=login%20ever=yes=1^3=plan=normal=1^5=gender=male=1^6=user_id=56405112=1^9=p_ab_id=2=1^10=p_ab_id_2=6=1^11=lang=zh=1; _gcl_au=1.1.2068989696.1697797468; PHPSESSID=56405112_5F6NwBp0ibUH6FuPbdq8P8dKXRrJiakY; device_token=d9e66683eee3c683671ce6dd45961915; _ga_MZ1NL4PHH0=GS1.1.1698243034.3.1.1698243060.0.0.0; __utmz=235335808.1698489562.713.11.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); QSI_S_ZN_5hF4My7Ad6VNNAi=v:0:0; _gid=GA1.2.73441192.1699788507; __utma=235335808.740189076.1647993924.1699936938.1699944133.746; _ga_ZPL8LPMDK3=GS1.1.1699945764.19.1.1699945776.0.0.0; __cf_bm=o75a_dxEYYH.70c_3Bc9l9kcnQ21g9q8qLYL8Cwim7E-1699947948-0-Ado847VODjuEbsuWh1d73rDUAUzc7NV4HSqPNkf4uDbtOq43lk4n0Iadho7XiG9BJqfShmOiyQ0SBKCLFsD9oftUNIV83XaDN3sMnjCMy9C9; __utmc=235335808; __utmt=1; cf_clearance=tKGeitoxfXwmrHstKob0LLLWi8vTys7CS5e3PwnggmI-1699948192-0-1-631cd866.945ac2d7.341d8786-0.2.1699948192; __utmb=235335808.54.10.1699944133; _ga_75BBYNYN9J=GS1.1.1699944133.496.1.1699948282.0.0.0; _ga=GA1.2.740189076.1647993924"
        }
        for illust_id in id_group]

    for url, add_headers in zip(urls, additional_headers):

        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36",
        }

        headers.update(add_headers)

        proxies = {
            "https": "127.0.0.1:7890"
        }

        response = requests.get(
                        url, headers=headers,
                        proxies=proxies,
                        timeout=4)

        # 获取 JSON 内容
        json_content = response.json()

        # 指定保存路径和文件名
        save_path = "E://vscode_clustter//code//github//Animate-Image-Bed//utils//crawler//test//jsons//re.json"

        # 将 JSON 内容保存到文件
        with open(save_path, "w", encoding="utf-8") as json_file:
            json.dump(json_content, json_file, ensure_ascii=False, indent=4)

        if response.status_code == 200:
            print(f"url: {url} , resp_code==200, collected Over !" )
            # 因为一个作品可能有多个img ，所以要进行归一化
            collected_urls = set()
            for url in response.json()["body"]:
                collected_urls.add(url["urls"]["original"])
        
        if collected_urls is not None:
            downloader.add(collected_urls)
        pbar.update()

===== collector start =====


collecting urls:   2%|▏         | 1/60 [00:00<00:53,  1.10it/s]

url: https://www.pixiv.net/ajax/illust/113130191/pages?lang=zh , resp_code==200, collected Over !


collecting urls:   3%|▎         | 2/60 [00:01<00:32,  1.79it/s]

url: https://www.pixiv.net/ajax/illust/113074184/pages?lang=zh , resp_code==200, collected Over !


...

url: https://www.pixiv.net/ajax/illust/113159949/pages?lang=zh , resp_code==200, collected Over !


collecting urls:  98%|█████████▊| 59/60 [00:27<00:01,  1.12s/it]

url: https://www.pixiv.net/ajax/illust/113021462/pages?lang=zh , resp_code==200, collected Over !


collecting urls: 100%|██████████| 60/60 [00:28<00:00,  2.14it/s]

url: https://www.pixiv.net/ajax/illust/113171326/pages?lang=zh , resp_code==200, collected Over !


> - ### Download Images

In [6]:
print("===== download start =====")
downloader.download()

===== download start =====
[INFO]: ===== downloader start =====


downloading / flow 306.21MB: 100%|██████████| 110/110 [00:29<00:00,  3.67it/s]

[INFO]: ===== downloader complete =====


306.20826625823975